In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import yaml
import numpy as np
import os.path as osp
from glob import glob 
from copy import deepcopy
from tqdm.notebook import tqdm

In [ ]:
import sys
sys.path.insert(0, '../../')
from mlcg.datasets import H5Dataset
from mlcg.utils import load_yaml, dump_yaml,makedirs


In [ ]:
!ls

# build a H5Dataset

In [ ]:
config = load_yaml('train_h5_1_10.yaml')['data']
config['partition_options'] = load_yaml(config['partition_options'])
config

In [ ]:
dataset = H5Dataset(**config)

In [ ]:
dataset._h5_root.keys()

In [ ]:
list(dataset._h5_root["CATH"].keys())

## make a new partition of the dataset

In [ ]:
def split_names(dataset, target, f):
    names = list(dataset._h5_root[target].keys())
    N = len(names)
    n_train = int(N*f)
    ids = np.arange(N)
    np.random.shuffle(ids)
    train_ids = ids[:n_train]
    test_ids = ids[n_train:]
    return {'train':[names[idx] for idx in train_ids], 'val':[names[idx] for idx in test_ids]}

In [ ]:
f = 0.8
partition_options = deepcopy(config['partition_options'])
data = ['OPEP', 'CATH']
for target in data:
    out = split_names(dataset, target, f)
    for k, ids in out.items():
        partition_options[k]['metasets'][target]['molecules'] = ids

In [ ]:
dump_yaml("./partition_settings_new.yaml", partition_options)